In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
base_path = "/content/drive/My Drive/yl_tez"

In [3]:
import os
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Yerelde Çalışıyor.")
    base_path = "."

Mounted at /content/drive


In [4]:
model_adi = "BestModelParameters"
model_yolu = os.path.join(base_path, model_adi)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_yolu)
model = AutoModelForCausalLM.from_pretrained(model_yolu)
model.to('cuda')
model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50262, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50262, bias=False)
)

In [6]:
import ipywidgets as widgets
from IPython.display import display, HTML

In [20]:
def generate_text(full_prompt_text, tokenizer, model, device, max_length=450):
    # Girdi metnini tokenize et ve attention mask oluştur
    encoded_input = tokenizer.encode(full_prompt_text, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(device)

    # Metin üretimi
    output_sequences = model.generate(
        input_ids=encoded_input,
        max_length=max_length,  # Cevap için ekstra uzunluk
        min_length=50,  # Minimum uzunluk belirleme
        temperature=0.7,  # Rastgelelik parametresi
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1
    )
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

    # Cevabı girdiden ayırma
    generated_response = generated_text[len(full_prompt_text):].strip()

    return generated_response
def apply_chat_template(message, doctor_title, doctor_speciality):
    doctor_info = f"<|DOCTOR_TITLE|> {doctor_title} <|SPECIALITY|> {doctor_speciality}\n"
    user_message = f"<|USER|> {message}\n"
    assistant_message = f"<|ASSISTANT|>"
    return f"{doctor_info}\n{user_message}\n{assistant_message}"

def test_et(tokenizer, model, device, pairs):
    # Ünvan ve uzmanlık dizilerini benzersiz ve case insensitive sıralı olarak oluşturma
    titles = sorted(list(set([title for title, speciality in unique_pairs_list])), key=lambda x: x.lower())
    specialities = sorted(list(set([speciality for title, speciality in unique_pairs_list])), key=lambda x: x.lower())

    while True:

        speciality_dropdown = widgets.Dropdown(
            options=specialities,
            description='Uzmanlık alanı:',
            disabled=False,
        )
        display(speciality_dropdown)

        girdi = input("Uzmanlık alanı seçin ve Enter'a basın (Çıkış için '0' yazın): \n")
        if girdi == '0':
            break

        selected_speciality = speciality_dropdown.value
        speciality_dropdown.disabled = True



        title_dropdown = widgets.Dropdown(
            options=titles,
            description='Doktor unvanı:',
            disabled=False,
        )
        display(title_dropdown)

        girdi = input("Doktor unvanı seçin ve Enter'a basın (Çıkış için '0' yazın): \n")
        if girdi == '0':
            break

        selected_title = title_dropdown.value
        title_dropdown.disabled = True

        # Kullanıcıdan soru girdisi alma
        prompt_text = input("Soru: \n")
        if prompt_text == '0':
            break

        # Tam soru metnini oluştur
        full_prompt_text = apply_chat_template(prompt_text, selected_title, selected_speciality)

        # Metin üretimi ve çıktının gösterilmesi
        generated_text = generate_text(full_prompt_text, tokenizer, model, device)
        display(HTML(f"<strong>{generated_text}</strong>"))


# Ünvan Alan Çiftleri

In [8]:
# Unique pairs
unique_pairs_list = [
    ('Dt.', 'dis-hekimi'),
    ('Dr. Dt.', 'dis-hekimi'),
    ('Uzm. Dr.', 'cocuk-sagligi-ve-hastaliklari'),
    ('Uzm. Dr.', 'cocuk-sagligi-ve-hastaliklari-cocuk-enfeksiyon-hastaliklari'),
    ('Uzm. Dr.', 'dermatoloji'),
    ('Prof. Dr.', 'genel-cerrahi'),
    ('Op. Dr.', 'kadin-hastaliklari-ve-dogum-ureme-endokrinolojisi-ve-infertilite'),
    ('Psk.', 'psikoloji'),
    ('Uzm. Dr.', 'dahiliye-ve-ic-hastaliklari'),
    ('Uzm. Dr. Dt.', 'dis-hekimi-oral-implantoloji'),
    ('Doç. Dr.', 'kadin-hastaliklari-ve-dogum-perinatoloji'),
    ('Uzm. Dr.', 'noroloji'),
    ('Op. Dr.', 'kalp-damar-cerrahisi'),
    ('Uzm. Dr.', 'algoloji'),
    ('Doç. Dr.', 'kalp-damar-cerrahisi'),
    ('Op. Dr.', 'goz-hastaliklari'),
    ('Klinik Psikolog ', 'psikoloji'),
    ('Prof. Dr.', 'psikiyatri'),
    ('Prof. Dr.', 'cocuk-sagligi-ve-hastaliklari-cocuk-kardiyolojisi'),
    ('Doç. Dr.', 'ortopedi-ve-travmatoloji'),
    ('Prof. Dr.', 'dahiliye-ve-ic-hastaliklari-endokrinoloji-ve-metabolizma-hastaliklari'),
    ('Doç. Dr.', 'kadin-hastaliklari-ve-dogum-ureme-endokrinolojisi-ve-infertilite'),
    ('Prof. Dr.', 'dis-hekimi'),
    ('Dr. Öğr. Üyesi', 'tibbi-genetik'),
    ('Prof. Dr.', 'kulak-burun-bogaz-hastaliklari'),
    ('Dr.', 'cocuk-sagligi-ve-hastaliklari'),
    ('Dr. Öğr. Üyesi', 'anestezi-ve-reanimasyon-algoloji-anestezi-ve-reanimasyon'),
    ('Fzt.', 'fizyoterapi'),
    ('Dyt.', 'diyetisyen'),
    ('Prof. Dr. Dt.', 'dis-hekimi-oral-diagnoz-ve-radyoloji'),
    ('Op. Dr.', 'ortopedi-ve-travmatoloji'),
    ('Uzm. Dr.', 'uroloji'),
    ('Uzm. Kl. Psk.', 'psikoloji'),
    ('Dr.', 'dis-hekimi-agiz-dis-ve-cene-cerrahisi'),
    ('Uzm. Dr.', 'radyoloji'),
    ('Op. Dr.', 'kadin-hastaliklari-ve-dogum'),
    ('Uzm. Dr.', 'psikiyatri'),
    ('Dr.', 'pratisyen-hekimlik'),
    ('Prof. Dr.', 'cocuk-sagligi-ve-hastaliklari-cocuk-immunolojisi-ve-alerji-hastaliklari'),
    ('Prof. Dr.', 'kadin-hastaliklari-ve-dogum'),
    ('Prof. Dr.', 'kardiyoloji'),
    ('Uzm. Dr.', 'gogus-hastaliklari'),
    ('Uzm. Psk.', 'psikoloji'),
    ('Prof. Dr.', 'goz-hastaliklari'),
    ('Yrd. Doç. Dr.', 'fiziksel-tip-ve-rehabilitasyon'),
    ('Uzm. Dr.', 'fiziksel-tip-ve-rehabilitasyon'),
    ('Op. Dr.', 'kulak-burun-bogaz-hastaliklari'),
    ('Uzm. Dr. Dt.', 'dis-hekimi-ortodonti'),
    ('Prof. Dr.', 'uroloji'),
    ('Psk. Dan.', 'psikolojik-danisman'),
    ('Dr. Dt.', 'dis-hekimi-ortodonti'),
    ('Doç. Dr.', 'gogus-cerrahisi'),
    ('Op. Dr.', 'plastik-rekonstruktif-ve-estetik-cerrahi'),
    ('Doç. Dr.', 'genel-cerrahi'),
    ('Doç. Dr.', 'dahiliye-ve-ic-hastaliklari-endokrinoloji-ve-metabolizma-hastaliklari'),
    ('Doç. Dr.', 'tibbi-genetik'),
    ('Uzm. Dr.', 'endokrinoloji-ve-metabolizma-hastaliklari'),
    ('Prof. Dr.', 'dis-hekimi-agiz-dis-ve-cene-cerrahisi'),
    ('Prof. Dr. Dt.', 'dis-hekimi-periodontoloji'),
    ('Prof. Dr.', 'kadin-hastaliklari-ve-dogum-ureme-endokrinolojisi-ve-infertilite'),
    ('Dr. Öğr. Üyesi', 'psikiyatri'),
    ('Prof. Dr.', 'ortopedi-ve-travmatoloji'),
    ('Op. Dr.', 'cocuk-cerrahisi'),
    ('Uzm. Dr.', 'kardiyoloji'),
    ('Dr.', 'aile-hekimligi'),
    ('Prof. Dr.', 'gastroenteroloji'),
    ('Uzm. Dt.', 'dis-hekimi-agiz-dis-ve-cene-cerrahisi'),
    ('Doç. Dr.', 'beyin-ve-sinir-cerrahisi'),
    ('Dt.', 'dis-hekimi-agiz-dis-ve-cene-cerrahisi'),
    ('Prof. Dr.', 'gogus-cerrahisi'),
    ('Psikoterapist', 'psikoloji'),
    ('Dr. Dt.', 'dis-hekimi-pedodonti'),
    ('Uzm. Dr.', 'beyin-ve-sinir-cerrahisi'),
    ('Doç. Dr.', 'cocuk-psikiyatrisi'),
    ('Op. Dr.', 'genel-cerrahi'),
    ('Uzm. Dt.', 'dis-hekimi'),
    ('Dr. Öğr. Üyesi', 'kulak-burun-bogaz-hastaliklari'),
    ('Dr. Dt.', 'dis-hekimi-periodontoloji'),
    ('Aile Danışmanı', 'aile-danismani'),
    ('Doç. Dr.', 'noroloji'),
    ('Uzm. Dr.', 'anestezi-ve-reanimasyon'),
    ('Dr. Dt.', 'dis-hekimi-agiz-dis-ve-cene-cerrahisi'),
    ('Doç. Dr.', 'psikiyatri'),
    ('Prof. Dr.', 'noroloji-noroloji-algoloji'),
    ('Uzm. Dr.', 'medikal-estetik-sertifikali-tip-doktoru'),
    ('Prof. Dr.', 'dahiliye-ve-ic-hastaliklari-gastroenteroloji'),
    ('Dr. Öğr. Üyesi', 'kadin-hastaliklari-ve-dogum'),
    ('Dr.', 'psikiyatri'),
    ('Dt.', 'dis-hekimi-pedodonti'),
    ('Uzm. Dr.', 'acil-tip'),
    ('Uzm. Dr.', 'genel-cerrahi'),
    ('Uzm. Dr.', 'dis-hekimi-periodontoloji'),
    ('Veteriner Hekim', 'veteriner'),
    ('Dr. Öğr. Üyesi', 'dis-hekimi-ortodonti'),
    ('Prof. Dr. Dt.', 'dis-hekimi-ortodonti'),
    ('Psk.', 'psikoloji-aile-danismani-psikolog'),
    ('Prof. Dr.', 'kalp-damar-cerrahisi'),
    ('Prof. Dr.', 'dermatoloji'),
    ('Dr.', 'kalp-damar-cerrahisi'),
    ('Çocuk Gelişim Uzmanı', 'cocuk-gelisim-uzmani'),
    ('Uzm. Dr.', 'dahiliye-ve-ic-hastaliklari-endokrinoloji-ve-metabolizma-hastaliklari'),
    ('Op. Dr.', 'beyin-ve-sinir-cerrahisi'),
    ('Doç. Dr.', 'dahiliye-ve-ic-hastaliklari-gastroenteroloji'),
    ('Uzm. Dr.', 'cocuk-psikiyatrisi'),
    ('Prof. Dr.', 'beyin-ve-sinir-cerrahisi'),
    ('Op. Dr.', 'genel-cerrahi-proktoloji'),
    ('Op. Dr.', 'kadin-hastaliklari-ve-dogum-perinatoloji'),
    ('Uzm. Dt.', 'dis-hekimi-restoratif-dis-tedavisi'),
    ('Op. Dr.', 'uroloji'),
    ('Dt.', 'dis-hekimi-oral-implantoloji'),
    ('Uzm. Dr.', 'enfeksiyon-hastaliklari-ve-klinik-mikrobiyoloji'),
    ('Prof. Dr.', 'cocuk-cerrahisi'),
    ('Doç. Dr.', 'anatomi'),
    ('Dr.', 'akupunktur'),
    ('Doç. Dr.', 'dis-hekimi-pedodonti'),
    ('Uzm. Dr.', 'tibbi-onkoloji-dahiliye-ve-ic-hastaliklari'),
    ('Uzm. Psk.', 'psikoterapi'),
    ('Prof. Dr.', 'dahiliye-ve-ic-hastaliklari-nefroloji'),
    ('Dil ve Konuşma Terapisti', 'dil-konusma-bozuklari-b-aile-danismani'),
    ('Uzm. Dr.', 'tibbi-patoloji'),
    ('Prof. Dr.', 'cocuk-sagligi-ve-hastaliklari-cocuk-norolojisi'),
    ('Prof. Dr.', 'tibbi-biyokimya'),
    ('Prof. Dr.', 'anestezi-ve-reanimasyon-algoloji-anestezi-ve-reanimasyon'),
    ('Uzm. Dr.', 'radyasyon-onkolojisi'),
    ('Uzm. Dr.', 'anestezi-ve-reanimasyon-algoloji-anestezi-ve-reanimasyon'),
    ('Dr.', 'medikal-estetik-sertifikali-tip-doktoru'),
    ('Op. Dr.', 'cocuk-cerrahisi-cocuk-cerrahisi-cocuk-urolojisi'),
    ('Dr.Öğr.Üyesi', 'psikoloji'),
    ('Prof. Dr.', 'plastik-rekonstruktif-ve-estetik-cerrahi'),
    ('Doç. Dr.', 'kulak-burun-bogaz-hastaliklari'),
    ('Doç. Dr.', 'romatoloji-dahiliye-ve-ic-hastaliklari'),
    ('Prof. Dr.', 'tibbi-farmakoloji'),
    ('Dt.', 'dis-hekimi-koruyucu-dis-hekimlig'),
    ('Prof. Dr.', 'cocuk-cerrahisi-cocuk-cerrahisi-cocuk-urolojisi'),
    ('Doç. Dr. Dt.', 'dis-hekimi-agiz-dis-ve-cene-cerrahisi'),
    ('Dr.', 'fiziksel-tip-ve-rehabilitasyon'),
    ('Prof. Dr.', 'kadin-hastaliklari-ve-dogum-jinekolojik-onkoloji'),
    ('Doç. Dr.', 'plastik-rekonstruktif-ve-estetik-cerrahi-el-cerrahisi-ve-mikrocerrahi-plastik'),
    ('Uzm. Dr.', 'akupunktur'),
    ('Uzm. Dr.', 'cocuk-cerrahisi'),
    ('Dr. Öğr. Üyesi', 'beyin-ve-sinir-cerrahisi'),
    ('Prof. Dr.', 'dahiliye-ve-ic-hastaliklari-immunoloji-ve-alerji-hastaliklari'),
    ('Dr.', 'anatomi'),
    ('Prof. Dr.', 'cocuk-sagligi-ve-hastaliklari-cocuk-nefrolojisi'),
    ('Uzm. Dr.', 'pratisyen-hekimlik'),
    ('Yrd. Doç. Dr.', 'kardiyoloji'),
    ('Prof. Dr.', 'dahiliye-ve-ic-hastaliklari-hematoloji'),
    ('Prof. Dr.', 'noroloji'),
    ('Prof. Dr.', 'enfeksiyon-hastaliklari-ve-klinik-mikrobiyoloji'),
    ('Doç. Dr.', 'cocuk-sagligi-ve-hastaliklari-cocuk-norolojisi'),
    ('Doç. Dr.', 'kardiyoloji'),
    ('Uzm. Dr. Dt.', 'dis-hekimi-restoratif-dis-tedavisi'),
    ('Dr.', 'dermatoloji'),
    ('Prof. Dr.', 'radyasyon-onkolojisi'),
    ('Uzm. Dt.', 'dis-hekimi-dis-protezi-uzmani'),
    ('Doç. Dr.', 'radyoloji-girisimsel-radyoloji'),
    ('Prof. Dr. Dt.', 'dis-hekimi-dis-protezi-uzmani'),
    ('Prof. Dr.', 'cocuk-sagligi-ve-hastaliklari-cocuk-hematoloji'),
    ('Dr.', 'acil-tip'),
    ('Uzm. Dr.', 'aile-hekimligi'),
    ('Prof. Dr.', 'cocuk-sagligi-ve-hastaliklari-cocuk-gogus-hastaliklari'),
    ('Uzm. Dr.', 'gogus-cerrahisi'),
    ('Prof. Dr.', 'fiziksel-tip-ve-rehabilitasyon-fiziksel-tip-ve-rehabilitasyon-romatoloji'),
    ('Dr.Öğr.Üyesi', 'diyetisyen'),
    ('Uzm. Dr.', 'spor-hekimligi'),
    ('Uzm. Dr.', 'cocuk-sagligi-ve-hastaliklari-cocuk-norolojisi'),
    ('Uzm. Dyt.', 'diyetisyen'),
    ('Doç. Dr.', 'uroloji'),
    ('Prof. Dr.', 'dis-hekimi-oral-implantoloji'),
    ('Prof. Dr.', 'dahiliye-ve-ic-hastaliklari-geriatri'),
    ('Dr. Öğr. Üyesi', 'tibbi-patoloji'),
    ('Prof. Dr.', 'radyoloji-uroradyoloji'),
    ('Uzm. Dr.', 'tibbi-genetik'),
    ('Dr. Dyt. ', 'diyetisyen'),
    ('Doç. Dr.', 'enfeksiyon-hastaliklari-ve-klinik-mikrobiyoloji'),
    ('Uzm. Psk. Dan.', 'psikolojik-danisman'),
    ('Dr. Öğr. Üyesi', 'cocuk-sagligi-ve-hastaliklari'),
    ('Prof. Dr.', 'romatoloji-dahiliye-ve-ic-hastaliklari'),
    ('Dil ve Konuşma Terapisti', 'dil-konusma-bozuklari-b'),
    ('Dr.', 'dahiliye-ve-ic-hastaliklari'),
    ('Uzm. Dt.', 'dis-hekimi-ortodonti'),
    ('Uzm. Dr. Dt.', 'dis-hekimi-dis-protezi-uzmani'),
    ('Uzm. Kl. Psk.', 'klinik-psikolog'),
    ('Doç. Dr.', 'kadin-hastaliklari-ve-dogum'),
    ('Uzm. Kl. Psk.', 'klinik-psikolog-psikoloji-aile-danismani-psikolog'),
    ('Uzm. Dr.', 'genel-cerrahi-cerrahi-onkoloji'),
    ('Uzm. Dr.', 'gastroenteroloji'),
    ('Prof. Dr.', 'genel-cerrahi-cerrahi-onkoloji'),
    ('Doç. Dr.', 'fiziksel-tip-ve-rehabilitasyon-fiziksel-tip-ve-rehabilitasyon-romatoloji'),
    ('Op. Dr.', 'gogus-cerrahisi'),
    ('Dr. Öğr. Üyesi', 'plastik-rekonstruktif-ve-estetik-cerrahi'),
    ('Uzm. Dr.', 'kalp-damar-cerrahisi'),
    ('Uzm. Dr.', 'ortopedi-ve-travmatoloji'),
    ('Prof. Dr.', 'cocuk-sagligi-ve-hastaliklari-cocuk-endokrinoloji'),
    ('Uzm. Psk.', 'psikoloji-aile-danismani-psikolog'),
    ('Prof. Dr.', 'anestezi-ve-reanimasyon'),
    ('Dr.', 'anestezi-ve-reanimasyon'),
    ('Prof. Dr.', 'dis-hekimi-periodontoloji'),
    ('Dr. Öğr. Üyesi', 'dis-hekimi-pedodonti'),
    ('Klinik Psikolog ', 'klinik-psikolog'),
    ('Doç. Dr.', 'genel-cerrahi-cerrahi-onkoloji'),
    ('Doç. Dr. Psk. Dan', 'psikolojik-danisman'),
    ('Doç. Dr. Dt.', 'dis-hekimi-ortodonti'),
    ('Prof. Dr.', 'fiziksel-tip-ve-rehabilitasyon'),
    ('Prof. Dr.', 'cocuk-sagligi-ve-hastaliklari'),
    ('Uzm. Dr.', 'dahiliye-ve-ic-hastaliklari-gastroenteroloji'),
    ('Dr. Öğr. Üyesi', 'dahiliye-ve-ic-hastaliklari'),
    ('Doç. Dr.', 'fiziksel-tip-ve-rehabilitasyon-fiziksel-tip-ve-rehabilitasyon-algoloji'),
    ('Doç. Dr.', 'goz-hastaliklari'),
    ('Doç. Dr.', 'dis-hekimi'),
    ('Dr. Öğr. Üyesi', 'ortopedi-ve-travmatoloji'),
    ('Op. Dr.', 'ortopedi-ve-travmatoloji-el-cerrahisi-ve-mikrocerrahi'),
    ('Prof. Dr.', 'tibbi-onkoloji-dahiliye-ve-ic-hastaliklari'),
    ('Uzm. Dr.', 'dis-hekimi-ortodonti'),
    ('Uzm. Dr.', 'tibbi-biyokimya'),
    ('Uzm. Dt.', 'dis-hekimi-pedodonti'),
    ('Dr. Öğr. Üyesi', 'fiziksel-tip-ve-rehabilitasyon'),
    ('Prof. Dr.', 'gogus-hastaliklari'),
    ('Prof. Dr.', 'cocuk-sagligi-ve-hastaliklari-cocuk-romatolojisi'),
    ('Uzm. Dr.', 'nukleer-tip'),
    ('Uzm. Kl. Psk.', 'klinik-psikolog-psikoloji'),
    ('Prof. Dr.', 'kadin-hastaliklari-ve-dogum-perinatoloji'),
    ('Dr. Öğr. Üyesi', 'noroloji'),
    ('Dr. Dt.', 'dis-hekimi-dis-protezi-uzmani'),
    ('Vet.', 'veteriner'),
    ('Prof. Dr.', 'dahiliye-ve-ic-hastaliklari'),
    ('Op. Dr.', 'plastik-rekonstruktif-ve-estetik-cerrahi-el-cerrahisi-ve-mikrocerrahi-plastik'),
    ('Dr.', 'geleneksel-ve-tamamlayici-tip'),
    ('Doç. Dr.', 'aile-hekimligi'),
    ('Dr.', 'kulak-burun-bogaz-hastaliklari'),
    ('Dt.', 'dis-hekimi-dis-protezi-uzmani'),
    ('Dr.', 'veteriner'),
    ('Prof. Dr. Dt.', 'dis-hekimi-pedodonti'),
    ('Doç. Dr.', 'dermatoloji'),
    ('Dr.', 'genel-cerrahi'),
    ('Prof. Dr.', 'dahiliye-ve-ic-hastaliklari-immunoloji'),
    ('Yrd. Doç. Dr.', 'cocuk-cerrahisi-cocuk-cerrahisi-cocuk-urolojisi'),
    ('Uzm. Dr.', 'kadin-hastaliklari-ve-dogum'),
    ('Psikoterapist', 'psikoloji-psikoonkoloji'),
    ('Prof. Dr.', 'genel-cerrahi-gastroenteroloji-cerrahisi'),
    ('Dr. Öğr. Üyesi', 'cocuk-sagligi-ve-hastaliklari-cocuk-gastroenterolojisi'),
    ('Doç. Dr.', 'genel-cerrahi-gastroenteroloji-cerrahisi'),
    ('Dr. Öğr. Üyesi', 'kalp-damar-cerrahisi'),
    ('Prof. Dr.', 'dis-hekimi-ortodonti'),
    ('Dr. Öğr. Üyesi', 'kadin-hastaliklari-ve-dogum-ureme-endokrinolojisi-ve-infertilite'),
    ('Uzm. Dr.', 'goz-hastaliklari'),
    ('Prof. Dr. Dt.', 'dis-hekimi'),
    ('Dr. Psk.', 'psikoloji'),
    ('Uzm. Dr.', 'dis-hekimi-agiz-dis-ve-cene-cerrahisi'),
    ('Uzm. Dr. Dt.', 'dis-hekimi-endodonti'),
    ('Prof. Dr.', 'tibbi-genetik'),
    ('Doç. Dr.', 'cocuk-sagligi-ve-hastaliklari-cocuk-endokrinoloji'),
    ('Prof. Dr.', 'cocuk-sagligi-ve-hastaliklari-neonatoloji'),
    ('Doç. Dr. Dt.', 'dis-hekimi'),
    ('Uzm. Dr.', 'dahiliye-ve-ic-hastaliklari-nefroloji'),
    ('Dr. Psk. Dan.', 'psikoloji-aile-danismani-psikolog'),
    ('Dr. Öğr. Üyesi', 'dis-hekimi-periodontoloji'),
    ('Dt.', 'pedodonti'),
    ('Doç. Dr.', 'plastik-rekonstruktif-ve-estetik-cerrahi'),
    ('Psk. Dan.', 'psikoloji'),
    ('Uzm. Psk. Dan.', 'psikolojik-danisman-psikoloji'),
    ('Dr.', 'tibbi-genetik'),
    ('Pedagog', 'pedagoji'),
    ('Prof. Dr.', 'cocuk-psikiyatrisi'),
    ('Prof. Dr.', 'cocuk-cerrahisi-cocuk-cerrahisi-cocuk-urolojisi-cocuk-urolojisi-uroloji'),
    ('Yrd. Doç. Dr.', 'plastik-rekonstruktif-ve-estetik-cerrahi'),
    ('Prof. Dr.', 'ortopedi-ve-travmatoloji-el-cerrahisi-ve-mikrocerrahi'),
    ('Op. Dr.', 'kalp-damar-cerrahisi-damar-cerrahisi'),
    ('Uzm. Dr.', 'kulak-burun-bogaz-hastaliklari'),
    ('Uzm. Kl. Psk.', 'psikoloji-aile-danismani-psikolog'),
    ('Prof. Dr.', 'tibbi-patoloji'),
    ('Dr.', 'akupunktur-biorezonans'),
    ('Doç. Dr.', 'dis-hekimi-agiz-dis-ve-cene-cerrahisi'),
    ('Dr. Öğr. Üyesi', 'cocuk-cerrahisi'),
    ('Uzm. Dr.', 'noroloji-klinik-norofizyoloji'),
    ('Uzm. Dr.', 'histoloji-ve-embriyoloji'),
    ('Dr. Öğr. Üyesi', 'geleneksel-ve-tamamlayici-tip'),
    ('Prof. Dr.', 'cocuk-sagligi-ve-hastaliklari-cocuk-enfeksiyon-hastaliklari'),
    ('Dt.', 'dis-hekimi-ortodonti'),
    ('Doç. Dr.', 'cocuk-sagligi-ve-hastaliklari-cocuk-immunolojisi-ve-alerji-hastaliklari'),
    ('Dr.', 'beyin-ve-sinir-cerrahisi'),
    ('Dr. Öğr. Üyesi', 'genel-cerrahi'),
    ('Prof. Dr.', 'genel-cerrahi-meme-cerrahisi'),
    ('Doç. Dr.', 'cocuk-sagligi-ve-hastaliklari-neonatoloji'),
    ('Doç. Dr.', 'dis-hekimi-dis-protezi-uzmani'),
    ('Uzm. Dr.', 'cocuk-sagligi-ve-hastaliklari-cocuk-kardiyolojisi'),
    ('Doç. Dr.', 'noroloji-klinik-norofizyoloji'),
    ('Psikoterapist', 'aile-danismani'),
    ('Dr. Dt.', 'dis-hekimi-endodonti'),
    ('Uzm. Dr.', 'gogus-hastaliklari-alerji-ve-gogus-hastaliklari'),
    ('Ergoterapist', 'ergoterapi'),
    ('Prof. Dr.', 'tibbi-onkoloji'),
    ('Op. Dr.', 'uroloji-androloji'),
    ('Uzm. Dr.', 'radyoloji-girisimsel-radyoloji'),
    ('Prof. Dr.', 'akupunktur'),
    ('Doç. Dr.', 'tibbi-onkoloji-dahiliye-ve-ic-hastaliklari'),
    ('Prof. Dr.', 'algoloji'),
    ('Dr. Öğr. Üyesi', 'acil-tip'),
    ('Doç. Dr.', 'radyasyon-onkolojisi'),
    ('Prof. Dr.', 'cocuk-sagligi-ve-hastaliklari-cocuk-genetik-hastaliklari'),
    ('Doç. Dr.', 'genel-cerrahi-proktoloji'),
    ('Prof. Dr. Dt.', 'dis-hekimi-oral-implantoloji'),
    ('Prof. Dr.', 'genel-cerrahi-endokrin-cerrahisi'),
    ('Prof. Dr.', 'cocuk-sagligi-ve-hastaliklari-cocuk-gastroenterolojisi'),
    ('Dr.', 'tibbi-patoloji'),
    ('Uzm. Dr.', 'ortopedi-ve-travmatoloji-el-cerrahisi-ve-mikrocerrahi'),
    ('Op. Dr.', 'kadin-hastaliklari-ve-dogum-jinekolojik-onkoloji'),
    ('Dr. Öğr. Üyesi Dt.', 'dis-hekimi-agiz-dis-ve-cene-cerrahisi'),
    ('Dr.', 'psikoloji'),
    ('Dr. Öğr. Üyesi', 'anatomi'),
    ('Dr. Öğr. Üyesi', 'gogus-hastaliklari'),
    ('Dr. Öğr. Üyesi', 'goz-hastaliklari'),
    ('Prof. Dr.', 'radyoloji'),
    ('Doç. Dr.', 'dahiliye-ve-ic-hastaliklari-immunoloji'),
    ('Dr. Öğr. Üyesi', 'radyasyon-onkolojisi'),
]
# Sort the unique pairs list alphabetically by speciality and then by title
unique_pairs_list = sorted(unique_pairs_list, key=lambda pair: (pair[1], pair[0]))

# Test

In [21]:
test_et(tokenizer, model, 'cuda', unique_pairs_list)

Dropdown(description='Uzmanlık alanı:', options=('acil-tip', 'aile-danismani', 'aile-hekimligi', 'akupunktur',…

Uzmanlık alanı seçin ve Enter'a basın (Çıkış için '0' yazın): 



Dropdown(description='Doktor unvanı:', options=('Aile Danışmanı', 'Dil ve Konuşma Terapisti', 'Doç. Dr.', 'Doç…

Doktor unvanı seçin ve Enter'a basın (Çıkış için '0' yazın): 

Soru: 
Dermotoloji olan tüm hastalara bakiyormusunuz yoksa sadece güzellik üzerine mi bana bilgi verebilirmisiniz tesekkür ederim


Dropdown(description='Uzmanlık alanı:', options=('acil-tip', 'aile-danismani', 'aile-hekimligi', 'akupunktur',…

Uzmanlık alanı seçin ve Enter'a basın (Çıkış için '0' yazın): 
0


# Örnekler

**Dt.
dis-hekimi
60 yaşında babamın dişlerini yaptırmak istiyoruz. yardımcı olabilir misiniz?
tabı ben patnostayım buyrun yardımcı olmak ısterım
Dt.
dis-hekimi
bir aydır ağzımda geçmeyen ufak bir yara var alt çenemde protezdenmi acaba ne yapmalıyım kötü bir hastalık olur diye korkmaya başladım lütfen beni aydınlatırmısınız meliha özkan
protez travmasına baglı epulıs gelısebılır yada patolojık bi durum olabılır bulunudugunuz yerde fakulte varsa cene hastalıkları yada cerrahısıne gorunmenız onerılır**

Dr. Dt.
dis-hekimi
Merhaba dun dis cekimi yapildiktan sonra bu sabah asiri kanama ile hekimime basvurdum.Pihtinin olustugunu yanliz kanamanin durmadigi gerekcesiyle dikis islemi uyguladi.Bugun herhangi bir kanamam veya agrim yok yalniz dikis bolgesinde mukusa benzer bir doku olustu.Tekrar gitmem gerekir mi?
Hayır.. rahat olun.normal şeyler yazmıssiniz.. yara yerinden mukus gibi dokunun cikmasi normaldir..


**Dr. Dt.
dis-hekimi
Merhaba.1,5 ay önce köprü dişlerim yapistirildi. O zaman ağrı vs yoktu.ancak son 10-15 gündür sanki bir şey içince köprünün altına gidiyor hissi var. Böyle düşünmeme sebep ise kendi disim ile köprü olan disin arasında diğer dişlere nazaran boşluk olması. Örneğin dış arası fırça kullanırken o bölgeye boydan boya girebiliyor fırça. Baslarda aralarda yapıştırıcı kaldığından hissetmedim belki de.ayrica köprü altındaki diğer dişlerimde kanal yapılanlarda da ağrı hissettim. Dişçi kontrol etti röntgen çekti herhangi bir sorun göremedi. Ayrıca baş ve kulak ağrısı var. 20lik dislerle de ilgili sıkıntı yok .daha önce dişciden kaynaklı bi sorundan dolayı güvenim azaldı dişçiye karşı. Dişçi emin bir şekilde koprunun altına gitmez dedi ama ben de bu sebeplerden dolayı emin olamadım.cevaplariniz için simdiden teşekkürler.
Öncelikle geçmiş olsun. Köprü yapımından sonra bu tarz hassasiyetler olabilir . Geçicidir. Kanallı dişlerinizde - kanalın yeni yapıldığını farz ederek söylüyorum- de ilk zamanlar, belki bir aya varan hassasiyet hissetmeniz normaldir. Röntgen çekildiğini ve sorun görülmediğini söylüyorsunuz. 20 liklerinizde de sorun görülmediğine göre belki baş kulak ağrısı için KBB ye görünmekte fayda olabilir diye düşünüyorum. Baş kulak ağrısı dişe bağlı olsa dişte ya da dişlerde hassasiyetten daha fazla ağrı şikayetiniz olurdu.**

Dr. Dt.
dis-hekimi
Merhaba doktor bey bnm çocuğumun üst dişlerinde 7ayligindan beri çürüme oluyor tamamen o dişinin beyaz yeri kalmaktadırlar curudu o kadar doktora götürdük ama bi tedavisini yapamadılar hepsi başka bişey diyo neden kaynaklanıyor olabilir
bazen lekeler çürük izlenimi verebiliyor.muayenede ortaya çıkar. uğrarsanız bilgilendirebiliriz...saygılarımızla...

**Dr. Dt.
dis-hekimi
Merhaba doktor bey bnm çocuğumun üst dişlerinde 7ayligindan beri çürüme oluyor tamamen o dişinin beyaz yeri kalmaktadırlar curudu o kadar doktora götürdük ama bi tedavisini yapamadılar hepsi başka bişey diyo neden kaynaklanıyor olabilir
ÖNCELİKLE GEÇMİŞ OLSUN BİR SÜRÜ SEBEBİ OLABİLİR ilaça bağlı renkleşme uzun süre biboron kullanımı çok fazla antibiyotik**